# CS446/519 Homework Assignment 2

load in the E. coli operon-operon network (gene regulatory network) data in edge-list format; display the first six lines of data as a sanity check. Use `read.table`.

In [30]:
network_edge_list_df  <- read.table("shared/ecolitfnet.txt",
                               sep="\t",
                               header=FALSE,
                               stringsAsFactors=FALSE)
head(network_edge_list_df)
dim(network_edge_list_df)

V1,V2
ADA,AIDB
ADIY,GADX
ALLR,ALLS
ARCA,BETI
ARCA,DPIA
ARCA,FNR


[1] 261   2

Make an igraph directed graph from the network; print a graph summary as a sanity check.  You will need to use `graph_from_data_frame` and `summary`.

In [3]:
library(igraph)
mygraph <- graph_from_data_frame(network_edge_list_df)
summary(mygraph)


Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union



IGRAPH DN-- 133 261 -- 
+ attr: name (v/c)


Obtain the counts of all three-vertex motifs in the graph. You will need to use the `motifs` function in igraph.

In [15]:
motif_counts_3vertexes <- motifs(mygraph, 3)
motif_counts_3vertexes

[1]   NA   NA  275   NA  125   39 1676   47   25   68    0    0    2   10    1
[16]    1

Get motif counts for 10,000 different randomizations of the network.  You will need to use `replicate`, `motifs`, and `rewire` with the `loops=FALSE` option.  Use niter=1000.

In [38]:
motif_counts_rand_3 <- replicate(10000, { motifs(rewire(mygraph, 
                            keeping_degseq(loops=FALSE, niter=1000)), 3) })


For each motif type, compute the mean and standard deviation of the counts of appearances of the motif in each of the 10,000 randomly rewired graphs. You will need to use the `apply` function.

In [39]:
motif_counts_rand_3_means <- apply(motif_counts_rand_3, 1, mean)
motif_counts_rand_3_sds <- apply(motif_counts_rand_3, 1, sd)

For each motif type, obtain the Z score for the number of appearances of the motif in the real network, vs. in the randomly rewired networks.

In [40]:
motif_counts_real_Z_3 <- (motif_counts_3vertexes - 
                          motif_counts_rand_3_means)/
                         (motif_counts_rand_3_sds)

Print out the mean number of FFLs on random networks, the number of FFLs on the real network, and the Z-score for the FFL count on the real network vs. the random networks. The FFL should be in the 8th element of the vector returned from `motifs`.

In [41]:
print(sprintf("Mean number of FFLs on random networks: %f", 
              motif_counts_rand_3_means[8]))
print(sprintf("Number of FFLs in the real network: %d", 
              motif_counts_3vertexes[8]))
print(sprintf("Z-score for FFL count on real network vs. random networks: %f", 
              motif_counts_real_Z_3[8]))


[1] "Mean number of FFLs on random networks: 57.247400"
[1] "Number of FFLs in the real network: 47"
[1] "Z-score for FFL count on real network vs. random networks: -1.112121"


Figure out the isomorphism class of the network shown in the assignment PDF.  You'll want to build an edge-list matrix using `rbind` and then an ingraph using `graph_data_frame`, and then use `graph.isoclass`. Print a summary of the graph.

In [33]:
edge.list.4pairs.testnet <- rbind(c(1,3),c(2,3),c(1,4),c(2,4))
graph.4pairs.testnet <- graph_from_data_frame(data.frame(edge.list.4pairs.testnet))
pairs.graph.isoclass <- graph.isoclass(graph.4pairs.testnet)
pairs.graph.isoclass
summary(graph.4pairs.testnet)

[1] 19

IGRAPH DN-- 4 4 -- 
+ attr: name (v/c)


Compute the number of appearances of this four-vertex motif in the real network, using `motifs` with `size=4`.

In [20]:
motif_counts_real_4pairs <- motifs(mygraph, 4)[pairs.graph.isoclass + 1]
print(sprintf("Count of the four-vertex DOR motif: %d", motif_counts_real_4pairs))

[1] "Count of the four-vertex DOR motif: 297"


Get the mean and standard deviation for the count of four-vertex motifs in 10,000 randomly rewired graphs.  You'll need to use `replicate` and `rewire`.

In [21]:
motif.counts.rand.4pairs <- replicate(10000, {graph.motifs(rewire(mygraph, keeping_degseq(loops=FALSE, niter=1000)), 4)})
motif.counts.rand.4pairs.mean <- mean(motif.counts.rand.4pairs[pairs.graph.isoclass + 1,])
motif.counts.rand.4pairs.sd <- sd(motif.counts.rand.4pairs[pairs.graph.isoclass + 1,])
motif.counts.rand.4pairs.Z <- (motif_counts_real_4pairs - 
                               motif.counts.rand.4pairs.mean)/
                               motif.counts.rand.4pairs.sd

print(sprintf("Average count of the four-vertex DOR motif for random networks: %f",
              motif.counts.rand.4pairs.mean))
print(sprintf("Standard deviation of the count of the four-vertex DOR motif for random networks: %f",
              motif.counts.rand.4pairs.sd))
print(sprintf("Z-score for DOR motif count in real vs. random networks:  %f",
              motif.counts.rand.4pairs.Z))

[1] "Count of the four-vertex DOR motif: 297"
[1] "Average count of the four-vertex DOR motif for random networks: 177.330700"
[1] "Standard deviation of the count of the four-vertex DOR motif for random networks: 37.732529"
[1] "Z-score for DOR motif count in real vs. random networks:  3.171516"
